In [1]:
import pandas as pd

df = pd.read_csv("../data/mtcars.csv")
print(df['qsec'].count())

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[["qsec"]] = scaler.fit_transform(df[["qsec"]])
print(df[df["qsec"] > 0.5]["qsec"].count())

32
9


In [2]:
import pandas as pd

df = pd.read_csv("../data/basic1.csv")
df_70 = df.loc[:len(df)*0.7-1]

# print(dir(pd.DataFrame))
df = df.sort_values(by=["f5"], ascending=False).reset_index()
val = df["f5"].loc[9]
df.loc[:9, "f5"] = val

print(df[df["age"] >= 80]["f5"].mean())

std_p = df_70["f1"].std()
med = df_70["f1"].median()
std_a = df_70["f1"].fillna(med).std()

print(abs(std_a - std_p))

age = df["age"]
m = age.mean()
std = age.std()

result = 0
for i in age:
    if m - std*1.5 <= i <= m + std*1.5:
        continue
    result += i
print(result)

62.49774712521738
3.2965018033960725
473.5


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 

In [4]:
df = pd.read_csv("../data/2_Train.csv")
train, test, y, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

train.drop("ID", axis=1, inplace=True)
y = y["Reached.on.Time_Y.N"]

id = test["ID"].reset_index(drop=True)
test.drop("ID", axis=1, inplace=True)
# train.info()
# y.info()
# print(train.shape, y.shape)
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
import sklearn
print(sklearn.ensemble.__all__)
c_col = train.select_dtypes(include="object").columns
n_col = train.select_dtypes(exclude="object").columns

# for col in c_col:
#     a = set(train[col].unique()) - set(test[col].unique())
#     b = set(train[col].unique()) - set(test[col].unique())
#     if not a and not b:
#         continue
#     else:
#         print(a, b)

# for col in n_col:
#     print(col, train[col].describe())

encoder = LabelEncoder()
scaler = MinMaxScaler()

for col in c_col:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])
    
train[n_col] = scaler.fit_transform(train[n_col])
test[n_col] = scaler.transform(test[n_col])

train, val, y, val_y = train_test_split(train, y, test_size=0.15, random_state=42)

models = [AdaBoostClassifier(random_state=40), GradientBoostingClassifier(random_state=42)]
for model in models:
    model.fit(train, y)
    
    y_pred = model.predict(val)
    ras = roc_auc_score(y_pred, val_y)
    acc = accuracy_score(y_pred, val_y)
    
    print(ras, acc)

test_prob = models[1].predict_proba(test)[:,1]
# print(test_prob.shape, id.shape)
result = pd.DataFrame({"ID":id, "Reached.on.Time_Y.N":test_prob})
print(result)


['BaseEnsemble', 'RandomForestClassifier', 'RandomForestRegressor', 'RandomTreesEmbedding', 'ExtraTreesClassifier', 'ExtraTreesRegressor', 'BaggingClassifier', 'BaggingRegressor', 'IsolationForest', 'GradientBoostingClassifier', 'GradientBoostingRegressor', 'AdaBoostClassifier', 'AdaBoostRegressor', 'VotingClassifier', 'VotingRegressor', 'StackingClassifier', 'StackingRegressor', 'HistGradientBoostingClassifier', 'HistGradientBoostingRegressor']
0.6738776259613127 0.6606060606060606


/Users/dhkim/miniconda3/envs/test_env/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.7238077923737039 0.6833333333333333
        ID  Reached.on.Time_Y.N
0     4733             0.453148
1     2040             0.993927
2     5114             0.404309
3     2361             0.993328
4     5996             0.369141
...    ...                  ...
2195   360             0.762136
2196  7832             0.264149
2197  9754             0.379545
2198  3751             0.444578
2199  4414             0.405841

[2200 rows x 2 columns]


In [5]:
import pandas as pd
df = pd.read_csv("../data/t1-data2.csv", index_col = "year")

med = df.loc["2022년"].median()
cnt = 0
for i in df.loc["2022년"]:
    if i > med:
        cnt += 1
print(cnt)

df = pd.read_csv("../data/t1-data1.csv")
df.dropna(inplace=True)
df = df.reset_index()
val = df.loc[:int(len(df)*0.6)]["f1"].quantile(0.75)
print(int(val - (val % 1)))

df = pd.read_csv("../data/t1-data1.csv")
print(df.isna().sum().sort_values().index[-1])

50
77
f1


In [6]:
import pandas as pd

train = pd.read_csv("../data/t2-1-train.csv")
test = pd.read_csv("../data/t2-1-test.csv")

y = train["TravelInsurance"]
train = train.drop(["TravelInsurance", "id"], axis=1)

train["AnnualIncome"] = train["AnnualIncome"].fillna(train["AnnualIncome"].median())

id = test.pop("id").reset_index(drop=True)
test["AnnualIncome"] = test["AnnualIncome"].fillna(train["AnnualIncome"].median())
# print(train.shape, test.shape, id.shape, y.shape)

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

c_col = train.select_dtypes(include="object").columns
n_col = train.select_dtypes(exclude="object").columns

scaler = MinMaxScaler()
train[n_col] = scaler.fit_transform(train[n_col])
test[n_col] = scaler.transform(test[n_col])

map_col = []
for col in c_col:
    a = set(train[col].unique())
    b = set(test[col].unique())

    if a == b:
        map_col.append(col)
        continue
    # print(col, a, b)
# print(train.shape, test.shape)

# print(help(pd.get_dummies))
train = pd.get_dummies(train, columns=map_col, drop_first=True)
test = pd.get_dummies(test, columns=map_col, drop_first=True)
train = pd.get_dummies(train, columns=["Employment Type"])
test = pd.get_dummies(test, columns=["Employment Type"], drop_first=True)

# print(set(train.columns) - set(test.columns))
# print(set(test.columns) - set(train.columns))

train, val, y, y_val = train_test_split(train, y, test_size=0.15, random_state=4)
# print(train.shape, val.shape, y.shape, y_val.shape)

models = [GradientBoostingClassifier(random_state=42), AdaBoostClassifier(random_state=42), RandomForestClassifier(random_state=42)]

for model in models:
    model.fit(train, y)
    predict = model.predict(val)
    ras = roc_auc_score(predict, y_val)
    ac = accuracy_score(predict, y_val)
    print(ras, ac)

model = GradientBoostingClassifier(random_state=42)
model.fit(train, y)
result = pd.DataFrame({"id":id, "TravelInsurance":model.predict_proba(test)[:,1]})
print(result.head())


0.850204081632653 0.8348214285714286
0.8351992536318805 0.8080357142857143
0.7573669918535654 0.78125
   id  TravelInsurance
0   0         0.342776
1   1         0.174348
2   2         0.027579
3   3         0.952573
4   4         0.038030


/Users/dhkim/miniconda3/envs/test_env/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [7]:
import pandas as pd

df = pd.read_csv("../data/high_blood_pressure.csv")

df["target"] = df["bp_post"] - df["bp_pre"]
print(round(df["target"].mean(), 2))

from scipy import stats
# print(help(stats.ttest_rel))
t, p = stats.ttest_rel(df["bp_post"], df["bp_pre"], alternative='less')
print(round(t, 4))
print(round(p, 4))

-6.12
-3.0002
0.0016


In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/Titanic_train.csv")

from scipy.stats import chi2_contingency
# print(dir(pd))

df_group = pd.crosstab(df["Sex"], df["Survived"])
result = chi2_contingency(df_group)
print(round(result.statistic, 3))

from statsmodels.api import Logit
formula = "Survived ~ Sex + SibSp + Parch + Fare"
# print(dir(Logit.from_formula))
result = Logit.from_formula(formula, df).fit()
print(round(result.params["Parch"], 3))

odds = np.exp(result.params["SibSp"])
print(round(odds, 3))

260.717
Optimization terminated successfully.
         Current function value: 0.482065
         Iterations 6
-0.201
0.702


In [9]:
import pandas as pd

df = pd.read_csv("../data/basic1.csv")

val = abs(df["age"].quantile(0.75) - df["age"].quantile(0.25))
print(int(val - val % 1))

df = pd.read_csv("../data/fb.csv")
# df.info()
df["target"] = (df["loves"] + df["wows"]) / df["reactions"]
cond1 = df["target"] > 0.4
cond2 = df["target"] < 0.5
cond3 = df["type"] == "video"

print(len(df[cond1 & cond2 & cond3]))

df = pd.read_csv("../data/nf.csv")
df.dropna(subset=["date_added"], inplace=True)
df["year"] = df["date_added"].str.split(" ").str[-1].str.strip()
df["month"] = df["date_added"].str.split(" ").str[0].str.strip()

# df.head()
cond1 = df["year"] == "2018"
cond2 = df["month"] == "January"
cond3 = df["country"] == "United Kingdom"
print(len(df[cond1 & cond2 & cond3]))

50
90
6


In [10]:
import pandas as pd
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
y = train.pop("Segmentation")

train = train.drop("ID", axis=1)
id = test.pop("ID").reset_index(drop=True)

# print(train.shape, y.shape, test.shape)

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score

c_col = train.select_dtypes(include="object").columns
n_col = train.select_dtypes(exclude="object").columns

# for col in c_col:
#     a = set(train[col])
#     b = set(test[col])
#     if a != b:
#         print(a, b)

scaler = MinMaxScaler()
encoder = LabelEncoder()

train[n_col] = scaler.fit_transform(train[n_col])
test[n_col] = scaler.transform(test[n_col])

for col in c_col:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])
    
train, val, y, y_val = train_test_split(train, y, test_size=0.15, random_state=42)
# print(train.shape, y.shape, val.shape, y_val.shape)

models = [RandomForestClassifier(random_state=42), AdaBoostClassifier(random_state=42), GradientBoostingClassifier(random_state=42)]

# print(help(f1_score))
for model in models:
    model.fit(train, y)
    pred = model.predict(val)
    f1_s = f1_score(pred, y_val, average="macro")
    acc = accuracy_score(pred, y_val)
    print(f1_s, acc)

result = pd.DataFrame({"ID":id, "Segmentation":model.predict(test)})
print(result)

0.4989795481593646 0.508
0.4971265300879215 0.516


/Users/dhkim/miniconda3/envs/test_env/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.5288451378801535 0.544
          ID  Segmentation
0     458989             1
1     458994             2
2     459000             3
3     459003             2
4     459005             2
...      ...           ...
2149  467950             1
2150  467954             4
2151  467958             1
2152  467961             2
2153  467968             4

[2154 rows x 2 columns]


In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/basic1.csv")

age = df["age"]

result = 0
cnt = 0
for i in age:
    if i % 1 != 0:
        result += np.floor(i) + np.ceil(i) + np.trunc(i)
        cnt += 1
print(result / cnt)

69.5


In [12]:
df = pd.read_csv("../data/basic1.csv")

# print(df.isna().sum() / len(df) * 100)

df.drop("f3", axis=1, inplace=True)

df_map = df.groupby("city")["f1"].median()
# print(help(df.fillna))
df["f1"] = df["f1"].fillna(df["city"].map(df_map))
print(df["f1"].mean())

65.52


In [13]:
import numpy as np

df = pd.read_csv("../data/t2-1-train.csv")
df.info()
sp_s = df["AnnualIncome"].skew()
sp_k = df["AnnualIncome"].kurt()
df["AnnualIncome_1"] = np.log1p(df["AnnualIncome"])

sp_l_s = df["AnnualIncome_1"].skew()
sp_l_k = df["AnnualIncome_1"].kurt()
print(sp_s, sp_k, sp_l_s, sp_l_k)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1490 non-null   int64  
 1   Age                  1490 non-null   int64  
 2   Employment Type      1490 non-null   object 
 3   GraduateOrNot        1490 non-null   object 
 4   AnnualIncome         1486 non-null   float64
 5   FamilyMembers        1490 non-null   int64  
 6   ChronicDiseases      1490 non-null   int64  
 7   FrequentFlyer        1490 non-null   object 
 8   EverTravelledAbroad  1490 non-null   object 
 9   TravelInsurance      1490 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 116.5+ KB
0.08269262230546265 -1.0113940872313758 -0.588070073691044 -0.5968175663162993
